<a href="https://colab.research.google.com/github/j-ranasinghe/Electricity-consumption-forecasting/blob/main/Generation_data_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import os
import time
import pandas as pd
from google.colab import files
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# Upload Datasets

In [2]:
consumption_data = pd.read_csv('/content/Demand for United States Lower 48 (region) hourly - UTC time.csv')   # read in the consumption data

In [33]:
holiday_data = pd.read_csv('/content/Holiday_data.csv')  # read in the holiday dataset

In [4]:
columns = ['date']
combined_data = pd.DataFrame(columns = columns) 

In [5]:
Coal = pd.read_csv('/content/Coal.csv')
Coal.rename(columns={'megawatthours': 'Coal'}, inplace=True)  # read in the coal generation data

Wind = pd.read_csv('/content/Wind.csv')
Wind.rename(columns={'megawatthours': 'Wind'}, inplace=True) # read in the wind generation data

Hydro = pd.read_csv('/content/Hydro.csv')
Hydro.rename(columns={'megawatthours': 'Hydro'}, inplace=True) # read in the hydro generation data

Solar = pd.read_csv('/content/Solar.csv')
Solar.rename(columns={'megawatthours': 'Solar'}, inplace=True) # read in the solar generation data

Other = pd.read_csv('/content/Other.csv')
Other.rename(columns={'megawatthours': 'Other'}, inplace=True) # read in the other generation data

Petroleum = pd.read_csv('/content/Petroleum.csv')
Petroleum.rename(columns={'megawatthours': 'Petroleum'}, inplace=True) # read in the petroleum generation data

Natural_gas = pd.read_csv('/content/Natural gas.csv')
Natural_gas.rename(columns={'megawatthours': 'Natural_gas'}, inplace=True) # read in the natural gas generation data

Nuclear = pd.read_csv('/content/Nuclear.csv')
Nuclear.rename(columns={'megawatthours': 'Nuclear'}, inplace=True) # read in the nuclear generation data

In [6]:
combined_data = combined_data.merge(Coal, on='date', how='outer')
combined_data = combined_data.merge(Wind, on='date', how='outer')
combined_data = combined_data.merge(Hydro, on='date', how='outer')
combined_data = combined_data.merge(Solar, on='date', how='outer')
combined_data = combined_data.merge(Other, on='date', how='outer')
combined_data = combined_data.merge(Petroleum, on='date', how='outer')
combined_data = combined_data.merge(Natural_gas, on='date', how='outer')
combined_data = combined_data.merge(Nuclear, on='date', how='outer')

In [7]:
print(combined_data.head())

                  date   Coal   Wind  Hydro  Solar  Other  Petroleum  \
0  2018-07-01 05:00:00  45037   3533   3206      1   5226        183   
1  2018-07-01 06:00:00  96248   7410   4270     -1   6460       3181   
2  2018-07-01 07:00:00  94535   8814   4352      0   5983       2987   
3  2018-07-01 08:00:00  93672  15377  22257     35   7037       3058   
4  2018-07-01 09:00:00  89685  14390  21434     25   7050       3062   

   Natural_gas  Nuclear  
0        66791    58363  
1        92199    75818  
2        89001    75650  
3       101095    81700  
4        97000    81733  


In [8]:
print(consumption_data.head())

                  date  megawatthours
0  2015-07-01 05:00:00         162827
1  2015-07-01 06:00:00         335153
2  2015-07-01 07:00:00         333837
3  2015-07-01 08:00:00         398386
4  2015-07-01 09:00:00         388954


In [34]:
print(holiday_data.head())

      dates      Holiday    WeekDay  Month  Day  Year
0  7/4/2004  4th of July     Sunday      7    4  2004
1  7/4/2005  4th of July     Monday      7    4  2005
2  7/4/2006  4th of July    Tuesday      7    4  2006
3  7/4/2007  4th of July  Wednesday      7    4  2007
4  7/4/2008  4th of July     Friday      7    4  2008


In [10]:
consumption_data['date'] = pd.to_datetime(consumption_data['date'], utc=True, infer_datetime_format=True)

In [11]:
combined_data['Total'] = combined_data.sum(axis=1) #get total generation units

<ipython-input-11-e15e0f5fedc8>:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  combined_data['Total'] = combined_data.sum(axis=1) #get total generation units


In [12]:
#filter dataset to required range of data

start_date = '2018-07-01'
end_date = '2021-07-31'
mask = (combined_data['date'] >= start_date) & (combined_data['date'] <= end_date)
combined_data = combined_data.loc[mask]

In [13]:
combined_data.describe().round(2)

,Coal,Wind,Hydro,Solar,Other,Petroleum,Natural_gas,Nuclear,Total
count,27019.00,27019.00,27019.00,27019.00,27019.00,27019.00,27019.00,27019.00,27019.00
mean,102202.83,35273.73,31688.98,8475.12,8092.47,3513.71,160248.71,90156.36,439651.92
std,27329.89,14039.78,8398.10,9906.84,1621.07,1454.54,42209.67,9377.98,72889.24
min,41087.00,3533.00,3206.00,-9.00,5201.00,183.00,66791.00,52868.00,182340.00
25%,81587.50,24404.50,25600.00,116.00,7197.00,2481.50,130729.00,85498.00,391401.50
50%,100150.00,34630.00,31737.00,2627.00,7916.00,3301.00,152281.00,91825.00,429362.00
75%,121662.50,45119.50,37507.00,16781.00,8717.00,4233.50,179252.00,96572.00,473154.50
max,173523.00,82964.00,58025.00,40778.00,33996.00,14118.00,325855.00,104655.00,704183.00


In [14]:
combined_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27019 entries, 0 to 27018
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         27019 non-null  object
 1   Coal         27019 non-null  int64 
 2   Wind         27019 non-null  int64 
 3   Hydro        27019 non-null  int64 
 4   Solar        27019 non-null  int64 
 5   Other        27019 non-null  int64 
 6   Petroleum    27019 non-null  int64 
 7   Natural_gas  27019 non-null  int64 
 8   Nuclear      27019 non-null  int64 
 9   Total        27019 non-null  int64 
dtypes: int64(9), object(1)
memory usage: 2.3+ MB


In [15]:
combined_data['date'] = pd.to_datetime(combined_data['date'], utc=True, infer_datetime_format=True)

In [16]:
# Find NaNs and duplicates in df_energy

print('There are {} missing values or NaNs in combined_data.'
      .format(combined_data.isnull().values.sum()))

temp_energy = combined_data.duplicated(keep='first').sum()

print('There are {} duplicate rows in combined_data based on all columns.'
      .format(temp_energy))

There are 0 missing values or NaNs in combined_data.
There are 0 duplicate rows in combined_data based on all columns.


In [35]:
# Create a new column 'holiday' with value 1 for holidays, 0 otherwise
holiday_data['holiday'] = 1

In [36]:
# Drop the year, month, and day columns
holiday_data = holiday_data.drop(['Year', 'Month', 'Day','WeekDay','Holiday'], axis=1)

In [37]:
# Convert the 'date' column to pandas datetime objects
holiday_data['dates'] = pd.to_datetime(holiday_data['dates'], format='%m/%d/%Y')

# Convert the datetime objects to strings in 'yyyy-mm-dd' format
holiday_data['dates'] = holiday_data['dates'].dt.strftime('%Y-%m-%d')

In [38]:
#filter dataset to required range of data

start_date = '2018-07-01'
end_date = '2021-07-31'
mask = (holiday_data['dates'] >= start_date) & (holiday_data['dates'] <= end_date)
holiday_data = holiday_data.loc[mask]

# Merging

In [39]:
# Create a complete list of dates for the time period covered by the generation data
start_date = '2018-07-01'
end_date = '2021-07-31'
date_range = pd.date_range(start=start_date, end=end_date, freq='d')

# Create a new dataframe with the complete list of dates
complete_dates = pd.DataFrame({'dates': date_range})

In [40]:
holiday_data['dates'] = pd.to_datetime(holiday_data['dates'])

In [41]:
# Merge the holiday data with the complete dates on the date column
merged_holiday_data = pd.merge(complete_dates, holiday_data, on='dates', how='left')

In [42]:
# Fill all NaN values with 0
merged_holiday_data.fillna(0, inplace=True)

In [43]:
print(merged_holiday_data.head())

       dates  holiday
0 2018-07-01      0.0
1 2018-07-02      0.0
2 2018-07-03      0.0
3 2018-07-04      1.0
4 2018-07-05      0.0


In [44]:
# create a new column 'date' and fill it with the dates from 'datetime' column
combined_data['dates'] = pd.to_datetime(combined_data['date']).dt.date

In [45]:
combined_data['dates'] = pd.to_datetime(combined_data['dates'])

In [46]:
merged_holiday_data.dtypes

dates      datetime64[ns]
holiday           float64
dtype: object

In [47]:
# Merge the holiday data with the complete dates on the date column
merged_data = pd.merge(merged_holiday_data, combined_data, on='dates', how='left')

In [51]:
# Drop the year, month, and day columns
Generation_dataset = merged_data.drop(["Coal","Wind","Hydro","Solar","Other","Petroleum","Natural_gas","Nuclear","dates"], axis=1)

In [53]:
# Merge the two datasets on the 'date' column
Generation_dataset = pd.merge(Generation_dataset, consumption_data, on='date', how='left')

In [54]:
print(Generation_dataset.head())

   holiday                      date     Total  megawatthours
0      0.0 2018-07-01 05:00:00+00:00  182340.0       509702.0
1      0.0 2018-07-01 06:00:00+00:00  285585.0       412574.0
2      0.0 2018-07-01 07:00:00+00:00  281322.0       390539.0
3      0.0 2018-07-01 08:00:00+00:00  324231.0       366425.0
4      0.0 2018-07-01 09:00:00+00:00  314379.0       354843.0


# Save Dataset

In [55]:
Generation_dataset.to_csv('Generation_dataset_combined.csv', index=False)